## 使用PaddleX API实现化妆品分类

In [2]:
import paddlex as pdx

import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0' # 设置使用0号GPU卡（如无GPU，执行此代码后仍然会使用CPU训练模型）
os.environ['CPU_NUM'] = '4'

import matplotlib
matplotlib.use('Agg')

[WARNING] pycocotools is not installed, detection model is not available now.
[WARNING] pycocotools install: https://paddlex.readthedocs.io/zh_CN/develop/install.html#pycocotools


In [4]:
"""定义图像处理流程transforms

定义训练和验证过程中的图像处理流程，其中训练过程包括一些数据增强操作
"""

from paddlex.cls import transforms

train_transforms = transforms.Compose([
    transforms.RandomCrop(crop_size=224),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize()
])

eval_transforms = transforms.Compose([
    transforms.ResizeByShort(short_size=256),
    transforms.CenterCrop(crop_size=224),
    transforms.Normalize()
])

In [5]:
"""定义数据集Dataset

使用PaddleX内置的数据集读取器读取训练和验证数据集，并应用配置的图像处理流程
本示例采用ImageNet数据集格式，因此采用paddlex.datasets.ImageNet来加载数据集
"""

train_dataset = pdx.datasets.ImageNet(
    data_dir='makeup',
    file_list='makeup/train_list.txt',
    label_list='makeup/labels.txt',
    transforms=train_transforms,
    shuffle=True
)

eval_dataset = pdx.datasets.ImageNet(
    data_dir='makeup',
    file_list='makeup/val_list.txt',
    label_list='makeup/labels.txt',
    transforms=eval_transforms
)

2023-01-13 16:50:53 [INFO]	Starting to read file list from dataset...
2023-01-13 16:50:53 [INFO]	710 samples in file makeup/train_list.txt
2023-01-13 16:50:53 [INFO]	Starting to read file list from dataset...
2023-01-13 16:50:53 [INFO]	132 samples in file makeup/val_list.txt


In [6]:
"""训练模型

本示例使用MobileNetV3_large_ssld分类模型
"""

num_classes = len(train_dataset.labels)
model = pdx.cls.MobileNetV3_large_ssld(num_classes=num_classes)

In [7]:
model.train(
    num_epochs=10,
    train_dataset=train_dataset,
    train_batch_size=32,
    eval_dataset=eval_dataset,
    lr_decay_epochs=[4, 6, 8],
    save_interval_epochs=1,
    learning_rate=0.025,
    save_dir='output/mobilenetv3_large_ssld'
)

D:\Installation\Anaconda\lib\site-packages\paddle\fluid\layers\math_op_patch.py:336: UserWarning: D:\Installation\Anaconda\lib\site-packages\paddlex\cv\nets\mobilenet_v3.py:231
The behavior of expression A * B has been unified with elementwise_mul(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwise_mul(X, Y, axis=0) instead of A * B. This transitional warning will be dropped in the future.
  warnings.warn(


2023-01-13 16:51:19 [INFO]	Downloading MobileNetV3_large_x1_0_ssld_pretrained.tar from https://paddle-imagenet-models-name.bj.bcebos.com/MobileNetV3_large_x1_0_ssld_pretrained.tar


100%|██████████| 21717/21717 [00:07<00:00, 2880.07KB/s]


2023-01-13 16:51:27 [INFO]	Decompressing output/mobilenetv3_large_ssld\pretrain\MobileNetV3_large_x1_0_ssld_pretrained.tar...
2023-01-13 16:51:29 [INFO]	Load pretrain weights from output/mobilenetv3_large_ssld\pretrain\MobileNetV3_large_x1_0_ssld_pretrained.
2023-01-13 16:51:29 [WARNING]	[SKIP] Shape of pretrained weight output/mobilenetv3_large_ssld\pretrain\MobileNetV3_large_x1_0_ssld_pretrained/fc_weights doesn't match.(Pretrained: (1280, 1000), Actual: (1280, 7))
2023-01-13 16:51:29 [WARNING]	[SKIP] Shape of pretrained weight output/mobilenetv3_large_ssld\pretrain\MobileNetV3_large_x1_0_ssld_pretrained/fc_offset doesn't match.(Pretrained: (1000,), Actual: (7,))
2023-01-13 16:51:30 [INFO]	There are 268 varaibles in output/mobilenetv3_large_ssld\pretrain\MobileNetV3_large_x1_0_ssld_pretrained are loaded.
2023-01-13 16:51:47 [INFO]	[TRAIN] Epoch=1/10, Step=2/22, loss=1.853426, acc1=0.3125, acc5=0.875, lr=0.025, time_each_step=8.49s, eta=0:37:56
2023-01-13 16:51:57 [INFO]	[TRAIN] Epoch

100%|██████████| 5/5 [00:08<00:00,  1.75s/it]


...

2023-01-13 17:07:02 [INFO]	[EVAL] Finished, Epoch=8, acc1=0.878788, acc5=1.0 .
2023-01-13 17:07:02 [INFO]	Model saved in output/mobilenetv3_large_ssld\best_model.
2023-01-13 17:07:03 [INFO]	Model saved in output/mobilenetv3_large_ssld\epoch_8.
2023-01-13 17:07:03 [INFO]	Current evaluated best model in eval_dataset is epoch_8, acc1=0.8787878787878788
2023-01-13 17:07:16 [INFO]	[TRAIN] Epoch=9/10, Step=2/22, loss=0.358533, acc1=0.90625, acc5=1.0, lr=2.5e-05, time_each_step=4.84s, eta=0:3:41
2023-01-13 17:07:26 [INFO]	[TRAIN] Epoch=9/10, Step=4/22, loss=0.581773, acc1=0.8125, acc5=1.0, lr=2.5e-05, time_each_step=4.88s, eta=0:3:32
2023-01-13 17:07:35 [INFO]	[TRAIN] Epoch=9/10, Step=6/22, loss=0.447583, acc1=0.875, acc5=1.0, lr=2.5e-05, time_each_step=4.86s, eta=0:3:22
2023-01-13 17:07:44 [INFO]	[TRAIN] Epoch=9/10, Step=8/22, loss=0.394063, acc1=0.84375, acc5=1.0, lr=2.5e-05, time_each_step=4.88s, eta=0:3:13
2023-01-13 17:07:54 [INFO]	[TRAIN] Epoch=9/10, Step=10/22, loss=0.513263, acc1=0.81

100%|██████████| 5/5 [00:08<00:00,  1.69s/it]


2023-01-13 17:08:59 [INFO]	[EVAL] Finished, Epoch=9, acc1=0.863636, acc5=1.0 .
2023-01-13 17:08:59 [INFO]	Model saved in output/mobilenetv3_large_ssld\epoch_9.
2023-01-13 17:08:59 [INFO]	Current evaluated best model in eval_dataset is epoch_8, acc1=0.8787878787878788
2023-01-13 17:09:12 [INFO]	[TRAIN] Epoch=10/10, Step=2/22, loss=0.53302, acc1=0.75, acc5=1.0, lr=2.5e-05, time_each_step=4.86s, eta=0:1:46
2023-01-13 17:09:21 [INFO]	[TRAIN] Epoch=10/10, Step=4/22, loss=0.422941, acc1=0.84375, acc5=1.0, lr=2.5e-05, time_each_step=4.89s, eta=0:1:36
2023-01-13 17:09:31 [INFO]	[TRAIN] Epoch=10/10, Step=6/22, loss=0.344156, acc1=0.90625, acc5=1.0, lr=2.5e-05, time_each_step=4.9s, eta=0:1:27
2023-01-13 17:09:40 [INFO]	[TRAIN] Epoch=10/10, Step=8/22, loss=0.740964, acc1=0.71875, acc5=1.0, lr=2.5e-05, time_each_step=4.84s, eta=0:1:16
2023-01-13 17:09:49 [INFO]	[TRAIN] Epoch=10/10, Step=10/22, loss=0.481297, acc1=0.875, acc5=1.0, lr=2.5e-05, time_each_step=4.85s, eta=0:1:6
2023-01-13 17:09:58 [INF

100%|██████████| 5/5 [00:08<00:00,  1.72s/it]


2023-01-13 17:10:54 [INFO]	[EVAL] Finished, Epoch=10, acc1=0.871212, acc5=1.0 .
2023-01-13 17:10:54 [INFO]	Model saved in output/mobilenetv3_large_ssld\epoch_10.
2023-01-13 17:10:54 [INFO]	Current evaluated best model in eval_dataset is epoch_8, acc1=0.8787878787878788


In [8]:
"""模型预测"""

result = model.predict('makeup/mascara/27.jpeg', topk=1)

result

[{'category_id': 6, 'category': 'mascara', 'score': 0.89641726}]